## Passing workflows between functions

Demo to use workflows in functions for end to end image analysis

This is slightly advanced, where we need to define a custom function for image analysis.

When you have a custom function and want to use it in napari-lattice with the GUI or CLI, you will need to create a `.py` file

In [1]:
import os
import pyclesperanto_prototype as cle
from napari_workflows import Workflow
from napari_workflows import _io_yaml_v1 as io_yaml

cle.get_device()


<Intel(R) Iris(R) Xe Graphics on Platform: Intel(R) OpenCL Graphics (1 refs)>

In [2]:
#download sample data from https://zenodo.org/records/14903188
img_path = "./sample_data/RBC_tiny.czi"

#make dir if not exists
if not os.path.exists("./sample_data"):
    os.makedirs("./sample_data")

import urllib.request

url = "https://zenodo.org/records/14903188/files/RBC_tiny.czi?download=1"
if not os.path.exists(img_path):
    print("Downloading file")
    urllib.request.urlretrieve(url, img_path)
    print(f"Downloaded at : {img_path}")
else:
    print(f"File already exists at : {img_path}")

File already exists at : ./sample_data/RBC_tiny.czi


In [3]:
#absolute path for saving
save_path = "C:/Users/rajasekhar.p/napari_lattice/notebooks/sample_data"

#make dir if not exists
if not os.path.exists(save_path):
    os.makedirs(save_path)


Lets apply a workflow, where we perform 
- median filter
- binarise using otsu algorithm with threshold of 1000
- label each object using connected components labeling-
- measure each object using connected components

As we have multiple steps, we can define it in a function and then call this function in a workflow. 

We would like to save the deskewed image, segmented image and the measurements as a csv file, so we return all 3.

In [4]:
from skimage.measure import regionprops_table
import numpy as np
import pandas as pd 

def image_seg_measure(input_image):
    # Apply median filter
    median_filtered = cle.median_sphere(input_image, radius_x=2, radius_y=2, radius_z=2)

    # Apply Otsu thresholding
    binary_image = cle.threshold_otsu(median_filtered)

    # Apply connected components labeling
    labeled_image = cle.connected_components_labeling_box(binary_image)
    
    #convert labeled_image to np array as original image will be an opencl array
    labeled_image_np = np.array(labeled_image)
    
    #measure using regionprops_table
    props = regionprops_table(labeled_image_np, input_image,
                              properties=['label', 'bbox','area', 
                                          'intensity_mean'])
    #convert to pandas dataframe
    #props_df = pd.DataFrame(props)
    return (input_image, labeled_image_np, props)

from lls_core import LatticeData

image_seg_workflow = Workflow()

#Otsu thresholding
input_arg = "deskewed_image"
#define a simple gaussian filter
task_name = "measurement"


#Apply function
image_seg_workflow.set(task_name, image_seg_measure,  input_arg )

print(image_seg_workflow)

Workflow:
measurement <- (<function image_seg_measure at 0x000001F9238FC160>, 'deskewed_image')



In [5]:
#Before we run this, lets look at whats in the directory
os.listdir(save_path)

['gaussian_workflow.yaml', 'RBC_tiny.czi']

In [6]:
params_workflow = LatticeData(
  input_image=img_path,
  save_dir=save_path,
  workflow=image_seg_workflow,
)

params_workflow.save()

[INFO:2025-08-29 11:48:41,214] Processing File ./sample_data/RBC_tiny.czi
Timepoints:   0%|          | 0/1 [00:00<?, ?it/s][INFO:2025-08-29 11:48:42,038] Processing File <xarray.DataArray 'transpose-d511a424874269c0d63d068279594e59' (Z: 834, Y: 118,
                                                                X: 209)> Size: 41MB
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 68B 'LatticeLightsheet'
  * Z        (Z) float64 7kB 0.0 0.3 0.6 0.9 1.2 ... 249.0 249.3 249.6 249.9
  * Y        (Y) float64 944B 0.0 0.145 0.29 0.435 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 2kB 0.0 0.145 0.29 0.435 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000001F9755C1440>


INFO: blockdim levels (1) < subsamp levels (3): First-level block size (4, 256, 256) will be used for all levels
INFO: blockdim levels (1) < subsamp levels (3): First-level block size (4, 256, 256) will be used for all levels


Timepoints: 100%|██████████| 1/1 [00:13<00:00, 13.41s/it]


In [15]:
#list files in save_path
os.listdir(save_path)


['gaussian_workflow.yaml',
 'RBC_tiny.czi',
 'RBC_tiny_deskewed.h5',
 'RBC_tiny_deskewed.xml',
 'RBC_tiny_deskewed_1.h5',
 'RBC_tiny_deskewed_1.xml',
 'RBC_tiny_deskewed_output_2.csv']

In [10]:
csv_path = "C:/Users/rajasekhar.p/napari_lattice/notebooks/sample_data/RBC_tiny_deskewed_output_2.csv"
df = pd.read_csv(csv_path)

#apply series.explode
df.apply(pd.Series.explode)

,Unnamed: 0,time,channel,label,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,area,intensity_mean
0,0,T0,C0,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,[23 8 5 7 7 18 5 6 15 6 7 4 6 5 34 ...,[ 49 179 85 550 676 748 651 1072 1289 ...,[ 0 0 0 0 0 0 0 0 0 0 0 ...,[32 57 58 14 50 41 52 41 16 58 40 32 33 50 36 ...,[ 64 291 615 556 742 795 1118 1295 1290 ...,[ 22 38 209 5 49 9 209 186 1 35 23 ...,[ 669 47513 889527 128 48386 30...,[ 824.44012451 815.22802734 897.43670654 82...


- RBC_tiny_deskewed.h5 is the deskewed image
- RBC_tiny_deskewed_1.h5 is the label image
- RBC_tiny_deskewed_output_2.csv is the table with measurements


Troubleshooting

To inspect the components of workflows, you can use the `process_workflow` function to access the data. 

It returns a generator object, which can be accessed using the `next` function

As we were returning 2 images and a measurement table/dictionary, it will have 3 outputs

In [7]:
for slice in params_workflow.process_workflow():
    first_slice = slice
    break

In [8]:
slice

('slices',
 <generator object LatticeData.process_workflow.<locals>._generator at 0x0000020086544900>)

In [9]:
data_set = next(first_slice[1])
data_set

Timepoints:   0%|          | 0/1 [00:00<?, ?it/s][INFO:2025-08-29 11:37:29,403] Processing File <xarray.DataArray 'transpose-2f7dac7b3adef14da02a10aa2414f35b' (Z: 834, Y: 118,
                                                                X: 209)> Size: 41MB
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 68B 'LatticeLightsheet'
  * Z        (Z) float64 7kB 0.0 0.3 0.6 0.9 1.2 ... 249.0 249.3 249.6 249.9
  * Y        (Y) float64 944B 0.0 0.145 0.29 0.435 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 2kB 0.0 0.145 0.29 0.435 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x00000200D8185260>


ProcessedSlice(data=(array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.

In [ ]:
measurements = data_set.data[2]#.apply(Series.explode)

measurements